In [19]:
# -*- coding: utf-8 -*-
import pandas as pd, numpy as np, re
from pathlib import Path

# ===== 경로 =====
FIRST_PATH = "/Users/igangsan/Desktop/forest_added.csv"   # 기준: ctpv_nm, sgg_nm 포함
GEO_PATH   = "/Users/igangsan/Desktop/ML/실험/geo.xlsx"   # "서울특별시 종로구 : 37.57… N, 126.97… E"
OUT_MERGED = "/Users/igangsan/Desktop/forest_geo_added.csv"
OUT_UNMATCH= "/Users/igangsan/Desktop/ML/실험/12.csv"
OUT_GEO    = "/Users/igangsan/Desktop/ML/실험/geo.csv"

NBSP = "\xa0"

# ===== 유틸 =====
def z(s): 
    return "" if pd.isna(s) else str(s)

def squash_spaces(s: str) -> str:
    return re.sub(r"\s+", " ", z(s).replace(NBSP, " ").strip())

def remove_all_spaces(s: str) -> str:
    return re.sub(r"\s+", "", squash_spaces(s))

def normalize_ctpv(ctpv: str) -> str:
    base = remove_all_spaces(ctpv)
    mp = {
        "서울특별시":"서울특별시","서울":"서울특별시",
        "부산광역시":"부산광역시","부산":"부산광역시",
        "대구광역시":"대구광역시","대구":"대구광역시",
        "인천광역시":"인천광역시","인천":"인천광역시",
        "광주광역시":"광주광역시","광주":"광주광역시",
        "대전광역시":"대전광역시","대전":"대전광역시",
        "울산광역시":"울산광역시","울산":"울산광역시",
        "경기도":"경기도","경기":"경기도",
        "충청북도":"충청북도","충북":"충청북도",
        "충청남도":"충청남도","충남":"충청남도",
        "전라남도":"전라남도","전남":"전라남도",
        "경상북도":"경상북도","경북":"경상북도",
        "경상남도":"경상남도","경남":"경상남도",
        "강원도":"강원특별자치도","강원특별자치도":"강원특별자치도","강원":"강원특별자치도",
        "제주도":"제주특별자치도","제주특별자치도":"제주특별자치도","제주":"제주특별자치도",
        "전라북도":"전북특별자치도","전북특별자치도":"전북특별자치도","전북":"전북특별자치도",
        "세종특별자치시":"세종특별자치시","세종시":"세종특별자치시","세종":"세종특별자치시",
    }
    return mp.get(base, base)

def strip_tail_sgg(s: str) -> str:
    s2 = remove_all_spaces(s)
    return s2[:-1] if s2 and s2[-1] in {"시","군","구"} else s2

def drop_city_prefix(sgg_clean: str) -> str:
    if "시" in sgg_clean:
        i = sgg_clean.rfind("시")
        if i >= 0 and i < len(sgg_clean)-1:
            return sgg_clean[i+1:]
    return sgg_clean

def sgn(val: float, hemi: str | None) -> float:
    if hemi is None: 
        return val
    hemi = hemi.upper()
    return -abs(val) if hemi in ("S","W") else abs(val)

# ===== 1) 기준 데이터 로드 =====
df = pd.read_csv(FIRST_PATH, dtype=str)
df.columns = df.columns.str.lower()
if not {"ctpv_nm","sgg_nm"}.issubset(df.columns):
    raise ValueError("forest_added.csv에 'ctpv_nm', 'sgg_nm' 컬럼이 필요합니다.")

# ===== 2) geo.xlsx 파싱 =====
geo_raw = pd.read_excel(GEO_PATH, header=None, dtype=str)
rows = []
for line in geo_raw[0].dropna().astype(str):
    raw = line.replace(NBSP, " ").strip()
    parts = re.split(r"\s*[:：]\s*", raw, maxsplit=1)
    if len(parts) != 2: continue
    name_part, coord_part = squash_spaces(parts[0]), parts[1]
    toks = name_part.split(" ")

    if len(toks) == 1 and remove_all_spaces(toks[0]) in {"세종특별자치시","세종시","세종"}:
        ctpv_nm, sgg_nm = "세종특별자치시", "세종시"
    elif len(toks) >= 2:
        ctpv_nm, sgg_nm = toks[0], " ".join(toks[1:])
    else: continue

    nums = re.findall(r"([-+]?\d+(?:\.\d+)?)", coord_part)
    if len(nums) < 2: continue
    lat_val, lon_val = float(nums[0]), float(nums[1])
    lat_hemi = re.search(r"([NS])", coord_part, flags=re.I)
    lon_hemi = re.search(r"([EW])", coord_part, flags=re.I)
    lat = sgn(lat_val, lat_hemi.group(1) if lat_hemi else None)
    lon = sgn(lon_val, lon_hemi.group(1) if lon_hemi else None)

    rows.append({"ctpv_nm": ctpv_nm, "sgg_nm": sgg_nm, "latitude": lat, "longitude": lon})

geo = pd.DataFrame(rows)
geo.columns = geo.columns.str.lower()
if geo.empty:
    raise RuntimeError("geo.xlsx 파싱 결과가 비었습니다.")

# ===== 3) geo 키 확장 =====
exp_rows = []
for _, r in geo.iterrows():
    ctpv_std = normalize_ctpv(r["ctpv_nm"])
    sgg_raw  = remove_all_spaces(r["sgg_nm"])

    if ctpv_std == "세종특별자치시":
        sgg_variants = {"세종시"}
    else:
        base = {sgg_raw, strip_tail_sgg(sgg_raw), drop_city_prefix(sgg_raw)}
        if sgg_raw in {"종로구", "종로"}:
            base.update({"종로구","종로"})
        sgg_variants = {s for s in base if s}

    if "군위" in sgg_raw:
        for alt_ctpv in ["경상북도","대구광역시"]:
            exp_rows.append({"ctpv_std": alt_ctpv, "sgg_std": "군위군",
                             "latitude": r["latitude"], "longitude": r["longitude"]})

    for s in sgg_variants:
        exp_rows.append({"ctpv_std": ctpv_std, "sgg_std": s,
                         "latitude": r["latitude"], "longitude": r["longitude"]})

geoX = pd.DataFrame(exp_rows).drop_duplicates(subset=["ctpv_std","sgg_std"]).reset_index(drop=True)
geoX.to_csv(OUT_GEO, index=False, encoding="utf-8-sig")

geo_key_to_coord = {(row.ctpv_std+"|"+row.sgg_std): (float(row.latitude), float(row.longitude))
                    for _, row in geoX.iterrows()}

# ===== 4) 기준 데이터 키 생성 =====
base = pd.DataFrame(index=df.index.copy())
base["ctpv_std"] = df["ctpv_nm"].map(normalize_ctpv)
sgg_main = df["sgg_nm"].map(lambda x: remove_all_spaces(x if pd.notna(x) else ""))

is_sejong = base["ctpv_std"].eq("세종특별자치시")
sgg_main = np.where(is_sejong, "세종시", sgg_main)
sgg_main = pd.Series(sgg_main).replace({"종로":"종로구"})

base["key_main"]   = base["ctpv_std"] + "|" + sgg_main
base["key_tail"]   = base["ctpv_std"] + "|" + pd.Series(sgg_main).map(strip_tail_sgg)
base["key_nocity"] = base["ctpv_std"] + "|" + pd.Series(sgg_main).map(drop_city_prefix)

is_gunwi = pd.Series(sgg_main).str.contains("군위")
base["key_gw1"] = np.where(is_gunwi, "경상북도|군위군", "")
base["key_gw2"] = np.where(is_gunwi, "대구광역시|군위군", "")

# ===== 5) 좌표 매칭 =====
lat, lon = pd.Series([np.nan]*len(base)), pd.Series([np.nan]*len(base))

def put_if_found(i, key: str) -> bool:
    if key and key in geo_key_to_coord:
        la, lo = geo_key_to_coord[key]
        lat.at[i] = la; lon.at[i] = lo
        return True
    return False

for i in base.index:
    if put_if_found(i, base.at[i,"key_gw1"]): continue
    if put_if_found(i, base.at[i,"key_gw2"]): continue
    if put_if_found(i, base.at[i,"key_main"]): continue
    if put_if_found(i, base.at[i,"key_tail"]): continue
    if put_if_found(i, base.at[i,"key_nocity"]): continue

# ===== 6) 저장 =====
out = df.copy()
out["latitude"], out["longitude"] = lat, lon

Path(OUT_MERGED).parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_MERGED, index=False, encoding="utf-8-sig")

unmatched = out[out["latitude"].isna()].copy()
unmatched.to_csv(OUT_UNMATCH, index=False, encoding="utf-8-sig")

print({"rows_total": len(out),
       "matched": len(out)-len(unmatched),
       "unmatched": len(unmatched),
       "outputs": {"merged_csv": OUT_MERGED,
                   "unmatched_csv": OUT_UNMATCH,
                   "geo_debug_csv": OUT_GEO}})


{'rows_total': 3560, 'matched': 3560, 'unmatched': 0, 'outputs': {'merged_csv': '/Users/igangsan/Desktop/forest_geo_added.csv', 'unmatched_csv': '/Users/igangsan/Desktop/ML/실험/12.csv', 'geo_debug_csv': '/Users/igangsan/Desktop/ML/실험/geo.csv'}}
